In [1]:
 %matplotlib inline
import seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from BorealWeights import BorealWeightedProblem
from gradutil import *
from pyomo.opt import SolverFactory

## Initialize data

In [2]:
revenue, carbon, deadwood, HA = init_boreal()
opt = SolverFactory('glpk')

In [3]:
carbon_clean = carbon.dropna(axis=0, how='any')
HA_clean = HA.dropna(axis=0, how='any')
deadwood_clean = deadwood.dropna(axis=0, how='any')
revenue_clean = revenue.dropna(axis=0, how='any')

# Optimization

Problem formulation

### Carbon without Nans

Lets solve the problem with just one objective and using data without Nan-values

In [4]:
min(carbon_clean)

In [5]:
%%time
bproblem = BorealWeightedProblem(carbon_clean.values)
opt.solve(bproblem.model)

In [6]:
def print_solution(problem,data):
    res_dict = dict()
    for i in problem.model.I:
        for j in problem.model.J:
            res_dict[j] = res_dict.get(j,0) + int(problem.model.x[i,j].value)
    print('Handling, # of stands')        
    for key in res_dict:
        print("{:8} {}".format(list(data)[key], res_dict[key]))

In [7]:
print_solution(bproblem, carbon_clean)

So it looks like we really are able to solve this problem using the original data! (Without Nans)
And also the distribution of variables makes sense (Set aside storages carbon the most, so it is the most common management regime)

### Carbon where Nan:s replaced with BAUs

Lets replace Nan:s with the corresponding BAU values and lets try to solve the problem. All the single optimization tasks are maximizing, so in the final solution there should be no zeros anyway.

In [8]:
n_revenue = nan_to_bau(revenue)
n_carbon = nan_to_bau(carbon)
n_deadwood = nan_to_bau(deadwood)
n_ha = nan_to_bau(HA)

revenue_norm = new_normalize(n_revenue.values)
carbon_norm = new_normalize(n_carbon.values)
deadwood_norm = new_normalize(n_deadwood.values)
ha_norm = new_normalize(n_ha.values)

In [9]:
carbon_zeros = carbon_norm

In [10]:
%%time
zero_bproblem = BorealWeightedProblem(carbon_zeros)
opt.solve(zero_bproblem.model)

In [14]:
print_solution(zero_bproblem, carbon)

The solution are still quite the same than with the previous try. Good.

#### Verifying that solution looks reasonable

In [15]:
lst = values_to_list(zero_bproblem.model, n_carbon.values)

In [16]:
min(lst)

There is no zeros at all, so at least by that aspect the result looks rational.

In [17]:
max(lst)

In [18]:
np.min(carbon.dropna(axis=0, how='any').values)

In [19]:
np.max(carbon.dropna(axis=0, how='any').values)

## Solving all single objective optimization tasks

#### Solving carbon storage

In [20]:
carbon_bproblem = zero_bproblem

In [21]:
carbon_values = values_to_list(carbon_bproblem.model, n_carbon.values)
print("Minimum: {}, maximum: {}".format(min(carbon_values), max(carbon_values)))

In [22]:
np.max(carbon)

In [23]:
print_solution(carbon_bproblem, n_carbon)

In [24]:
this_bproblem = carbon_bproblem
sum([this_bproblem.model.x[i,j].value for i in this_bproblem.model.I for j in this_bproblem.model.J])

In [25]:
sum(carbon_values)

#### Solving HA

In [26]:
HA_zeros = ha_norm

In [27]:
%%time
HA_bproblem = BorealWeightedProblem(HA_zeros)
opt.solve(HA_bproblem.model)

In [28]:
HA_values = values_to_list(HA_bproblem.model, n_ha.values)
print("Minimum: {}, maximum: {}".format(min(HA_values), max(HA_values)))

In [29]:
print_solution(HA_bproblem, n_ha)

In [30]:
sum(HA_values)

These results make sense also. The values match to the ones attained in the previous notebook, which is great.

#### Solving deadwood

In [31]:
deadwood_zeros = deadwood_norm

In [32]:
%%time
deadwood_bproblem = BorealWeightedProblem(deadwood_zeros)
opt.solve(deadwood_bproblem.model)

In [33]:
deadwood_values = values_to_list(deadwood_bproblem.model, n_deadwood.values)
print("Minimum: {}, maximum: {}".format(min(deadwood_values), max(deadwood_values)))

In [34]:
print_solution(deadwood_bproblem, n_deadwood)

In [35]:
sum(deadwood_values)

Matching.

#### Solving Timber revenue

In [36]:
revenue_zeros = revenue_norm

In [37]:
%%time
revenue_bproblem = BorealWeightedProblem(revenue_zeros)
opt.solve(revenue_bproblem.model)

In [38]:
revenue_values = values_to_list(revenue_bproblem.model, n_revenue.values)
print("Minimum: {}, maximum: {}".format(min(revenue_values), max(revenue_values)))

In [39]:
print_solution(revenue_bproblem, n_revenue)

First of all, it would make sense that the BAU scheme should produce the most profit and so there should be more BAU handlings. Apparently this is not the case. The NT\*R schemes are prolonged versions of BAU, so probably they actually are able to produce bigger timbers and so more revenue. I think this still is correct result.

There were some stands with value 0.0, so lets check them:

In [40]:
rv = np.array(revenue_values)
revenue.iloc[rv == 0]

Apparently on some stands there is no way to make any profit, so it is ok that those are only zeros.

In [41]:
sum(revenue_values)

The final objective value is still the right one.

## Comparing optimizations to ones in the papers

The single objective results are documented in http://onlinelibrary.wiley.com/doi/10.1111/1365-2664.12790/full
so it is meaningful to compare our results to that one.

Results in the paper:
"The maximum capacity of the landscape 
- (i) to provide harvest revenues (NPV) was 250 M€ (average 5800 € ha−1),
- (ii) to store carbon was 4459 × 10³ MgC (average 10³ MgC ha−1), 
- (iii) for deadwood index was 218 150 m³ (average 5·1 m³ ha−1) and 
- (iv) for the combined habitat availability was 20 211 (no units) (average 0·47 ha−1)."

For us the correspondig values are:

In [45]:
print("(i) Harvest revenues {:.0f} M€".format(sum(revenue_values)/1000000))
print("(ii) Carbon storage {:.0f} x 10³ MgC".format(sum(carbon_values)/1e+3))
print("(iii) Deadwood index {:.0f} m3".format(sum(deadwood_values)))
print("(iv) Combined Habitat {:.0f}".format(sum(HA_values)))

There is still something weird with the data values given in the paper:


In [46]:
print('Total ha-1 calculated according to the values given in paper:')
print('-'*62)
print('Revenue/(average timber revenue/ha-1) = {}'.format(250000000/5800))
print('Carbon/(average carbon storage /ha-1) = {}'.format(4459*1000/103))
print('Deadwood/(average deadwood index/ha-1) = {}'.format(218150/5.1))
print('Combined habitat/(average habitat/ha-1) = {}'.format(20211/0.47))

Now all these values indicate that there should be ~43000 hectars in total. Paper still states that there were 68 700 hectars. I don't really know if that is a real problem regarding the optimization task, but it's still odd.

Lets draw the same pictures than in the paper, so we can see if there are some deviations there also.

In [47]:
%pylab inline
pylab.rcParams['figure.figsize'] = (15,12)

ind = list(revenue)
ind.append('Öptim.')

fig, ax = plt.subplots(2,2)
val = revenue.sum().values
val = np.append(val, sum(revenue_values))
ax[0,0].bar(ind, val)
ax[0,0].set_title('Timber Harvest Revenues')

val = carbon.sum().values
val = np.append(val, sum(carbon_values))
ax[0,1].bar(ind, val)
ax[0,1].set_title('Carbon storage')

val = deadwood.sum().values
val = np.append(val, sum(deadwood_values))
ax[1,0].bar(ind, val)
ax[1,0].set_title('Deadwood')

val = HA.sum().values
val = np.append(val, sum(HA_values))
ax[1,1].bar(ind, val)
ax[1,1].set_title('Habitat availability')



These really doesn't look like the same as in the paper. Something must be wrong with the way I am summing values without optimization.

In [51]:
from IPython.display import Image
Image('../images/articleOptims.png')

### Pay-off Table

In [52]:
data = np.dstack((n_revenue, n_carbon, n_deadwood, n_ha))
problems = [revenue_bproblem, carbon_bproblem, deadwood_bproblem, HA_bproblem]

In [53]:
payoff = [[np.sum(values_to_list(problems[j].model, data[:, :, i]))
           for i in range(np.shape(data)[-1])]
          for j in range(len(problems))]
z_ideal = np.max(payoff, axis=0)
z_nadir = np.min(payoff, axis=0)


In [54]:
z_ideal, z_nadir,

These look correct, so everything is fine